# Deterministic finite Automaton

In [1]:
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
class Automaton:
    """
    Class Automaton which simultaes how a deterministic finite accepter works
    """
    
    def __init__(self, q = None, sigma = None, delta = None, q0 = None, f = None):
        """
        Class initializar
        
        self.q receives all the states that conform the dfa
        
        self.sigma are all the variables defined in the dfa
        
        self.delta all are the operation defined, i.e- is a dictionary contating a tuple as key
        where the left values of the tuple is the current state and the right value the variable pass and the value obtain by
        accessing to with the key is to which state goes.
        
        self.q0 is the initial state
        
        self.f are all the final states
        
        self.actual_state is the state which the dfa finish after proccessing a word, if no word proccess the actual state is q0
        and it is reset at start proccessing a new word
        """
        self.q = q
        self.sigma = sigma
        self.delta = delta
        self.q0 = q0
        self.f = f
        self.actual_state = q0
        
    def load_from_file(self, path, delta_separator = "-"):
        """
        Function to read a file that contains an automaton
        
        path: path file
        """
        
        #dictionary to save the parts of the automaton
        variables = {"q": None,
             "sigma": None,
             "delta": None,
             "q0": None,
             "f": None    
        }

        with open(path) as file:
            #parsing the document
            for line, variable in zip(file, variables):
                    line = line.replace("{", "")
                    line = line.replace("}", "")
                    line = line.replace("\n", "")
                    line = line.replace(" ", "")
                    value = list(line.split(","))
                    variables[variable] = value
                    
            #conditional to verify there is only one initial state        
            if len(variables["q0"]) > 1 or len(variables["q0"]) == 0:
                raise Exception('There are more than one initial state')

            
            #getting delta
            variables["delta"] = [values.split(delta_separator) for values in variables["delta"]]
            
            #memory to check there in not repited transiction
            Memory = [(value[0],value[1]) for value in variables["delta"]]
            
            Aux_memory = []
            
            for var in Memory:
                if var not in Aux_memory:
                    Aux_memory.append(var)                    
                else:
                    raise TypeError("There is a transiction repited")
                    
            #Memory to check that within the transictions there is no variable with more than one variable
            Memory = [True for value in variables["delta"] if len(value[1]) > 1]
            
            if sum(Memory) >= 1:
                raise TypeError("At least One variable within a transictions contains more than 1 variable")
            
            
            
            variables["delta"] = {(value[0],value[1]): value[2] for value in variables["delta"]}
            
            #getting q0
            variables["q0"] = variables["q0"][0]
            
            #assigning the attributos of the class to the values obtained from the file
            self.q = variables["q"]
            self.sigma = variables["sigma"]
            self.delta = variables["delta"]
            self.q0 = variables["q0"] 
            self.f = variables["f"]
            self.actual_state = variables["q0"]
            
        #conditional to verify the dfa is valid
        if self.is_valid():
            pass
        else:
            raise Exception('Is not a valid automaton')
            
            
        
    def proccess_word(self, word):
        """
        Function to determine if a word belong to the language of the dfa
        """
        
        #restarting the actual state to the initial state
        self.actual_state = self.q0 
        
        #loop to make our dfa proccess the word
        for (letter) in str(word):
            try:
                #changing the current state of the dfa by using a tuple as a key for our dictionary of delta
                self.actual_state = self.delta[(self.actual_state,letter)]
            except:
                raise Exception('That transiction is not defined, so no valid word')
                
        if self.actual_state in self.f:
            print("Valid word")
            
        else:
            print("Unvalid word")
                 
                
    def __str__(self):
        cadena = "Grammar: \n q: {} \n sigma: {}, delta: {}, q0: {}, f: {}, current_sate: {}".format(
        self.q , self.sigma, self.delta, self.q0, self.f, self.actual_state) 
        return cadena
    
    def is_valid(self):
        
        """
        Funtion to validate whether the dfa is valid or not
        
        Uses the attributes of the class
        """
        
        #gets the tuples which are the deltas defined
        all_defined_oper = list(self.delta.keys())
        
        #get the values of the deltas
        all_to_go_states = list(self.delta.values())
        
        #gets the left value of the tuple
        all_states_to_receive = [state[0] for state in list(self.delta.keys())]
        
        #gets the right value of the tuple
        all_variables_to_receive = [state[1] for state in list(self.delta.keys())]
        
        
        #conditional to check if the final states are subsets of q, which is the variable containing all the states
        if set(self.f).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("States of F are not a subset of states q")
            
        
        #conditional to check if the inital state is a subsets of q, which is the variable containing all the states    
        if set([self.q0]).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("q0 is not a subset of states q")
        
        
        #checking if all the transictions of delta are subsets of q
        if set(all_to_go_states).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state resultant of a transiction is not a subset of states q")
        
        #checking if the left value of the tuple is subset of q and the right value is subset of sigma
        if set(all_states_to_receive).issubset(set(self.q)):
            pass
        
        else:
            raise TypeError("The state before a applying the transiction is not a subset of q")
            
        if set(all_variables_to_receive).issubset(set(self.sigma)):
            pass
        
        else:
            raise TypeError("The variable which is apply to the transiction is not a subset of sigma")
        
        
        #checking that each variable have defined a transition on each state
        for state in self.q:
            for variable in self.sigma:
                if (state,variable) in all_defined_oper:
                    pass
                else:
                    raise TypeError("At least one variable has not a defined transiction in a state")
                
        
        return 1 
    
        
              

## Implementation explanation

#### Explaining the parsing

```
for line, variable in zip(file, variables):
                    line = line.replace("{", "")
                    line = line.replace("}", "")
                    line = line.replace("\n", "")
                    line = line.replace(" ", "")
                    value = list(line.split(","))
                    variables[variable] = value
```

That code eliminates all the characters that are not neccesarry and then split the values of each line into a list, follow by saving the values on a dictionary

```
variables["delta"] = [values.split("-") for values in variables["delta"]]
variables["delta"] = {(value[0],value[1]): value[2] for value in variables["delta"]}
```

here we separate the values of delta into tuplet key and the state it goes after receiving that key

#### Explaining the method proccess_word

```
for (letter) in str(word):
            try:
                self.actual_state = self.delta[(self.actual_state,letter)]
            except:
                raise Exception('That transiction is not defined, so no valid word')
                
```

the loop works to get one character of the string one by one, and using try we check that the transictions is especified and since we know that our automata is valid thanks to the method is_valid(), we know that an error is because receive a letter that does not exist in sigma

```
if self.actual_state in self.f:
            print("Valid word")
            
        else:
            print("Unvalid word")
            
```

So, in the last state of the proccess words if is a subset of the set of final states, so is a valid word, in other way is not.

## Implementation Example

#### Creating an object of the class Automaton

In [3]:
Automata = Automaton()

#### Using one of its public method to read an automata from a file

In [4]:
Automata.load_from_file("data/automata.txt")

#### Accesing to its attributes

$q$

In [5]:
Automata.q

['q0', 'q1', 'q2', 'q3']

 $\delta$

In [6]:
Automata.delta

{('q0', '0'): 'q3',
 ('q0', '1'): 'q1',
 ('q1', '0'): 'q2',
 ('q1', '1'): 'q1',
 ('q2', '0'): 'q2',
 ('q2', '1'): 'q1',
 ('q3', '0'): 'q3',
 ('q3', '1'): 'q3'}

$\Sigma$

In [7]:
Automata.sigma

['0', '1']

$q_{0}$

In [8]:
Automata.q0

'q0'

$f$

In [9]:
Automata.f

['q2']

#### Examples of proccessing words

In [10]:
Automata.proccess_word("01")

Unvalid word


In [11]:
Automata.proccess_word("100000")

Valid word


#### Using the method str

In [12]:
str(Automata)

"Grammar: \n q: ['q0', 'q1', 'q2', 'q3'] \n sigma: ['0', '1'], delta: {('q0', '0'): 'q3', ('q0', '1'): 'q1', ('q1', '0'): 'q2', ('q1', '1'): 'q1', ('q2', '0'): 'q2', ('q2', '1'): 'q1', ('q3', '0'): 'q3', ('q3', '1'): 'q3'}, q0: q0, f: ['q2'], current_sate: q2"

### Implementation 2

In [13]:
Automata2 = Automaton()

In [14]:
Automata2.load_from_file("data/automata2.txt")

In [15]:
Automata2.sigma

['a', 'b']

In [16]:
Automata2.delta

{('A', 'a'): 'B',
 ('A', 'b'): 'A',
 ('B', 'a'): 'C',
 ('B', 'b'): 'A',
 ('C', 'a'): 'C',
 ('C', 'b'): 'A'}

In [17]:
Automata2.f

['C']

In [18]:
Automata2.proccess_word("ab")

Unvalid word


In [19]:
Automata2.proccess_word("aaaaaaa")

Valid word


# Implementation 3

In [20]:
Automata3 = Automaton()

In [21]:
Automata3.load_from_file("data/automata3.txt")

In [22]:
Automata3.sigma

['a', 'b']

In [23]:
Automata3.delta

{('0', 'a'): '1',
 ('0', 'b'): '2',
 ('1', 'a'): '3',
 ('1', 'b'): '2',
 ('3', 'a'): '3',
 ('3', 'b'): '4',
 ('4', 'a'): '2',
 ('4', 'b'): '4',
 ('2', 'a'): '2',
 ('2', 'b'): '2'}

In [24]:
Automata3.f

['4']

In [25]:
Automata3.proccess_word("aaaabbbbb")

Valid word


In [26]:
Automata3.proccess_word("aaaababbbb")

Unvalid word


# Implementation 4

In [27]:
Automata4 = Automaton()

In [28]:
Automata4.load_from_file("data/automata4.txt")

In [29]:
Automata4.sigma

['a', 'b']

In [30]:
Automata4.delta

{('0', 'a'): '1',
 ('0', 'b'): '3',
 ('1', 'a'): '1',
 ('1', 'b'): '2',
 ('3', 'a'): '2',
 ('3', 'b'): '4',
 ('4', 'a'): '2',
 ('4', 'b'): '4',
 ('2', 'a'): '5',
 ('2', 'b'): '5',
 ('5', 'a'): '5',
 ('5', 'b'): '5'}

In [31]:
Automata4.f

['2', '3']

In [32]:
Automata4.proccess_word("aaaab")

Valid word


In [33]:
Automata4.proccess_word("aaaabbbb")

Unvalid word


# Implementation 5

In [34]:
Automata5 = Automaton()

In [35]:
Automata5.load_from_file("data/automata5.txt")

In [36]:
Automata5.sigma

['a', 'b']

In [37]:
Automata5.delta

{('0', 'a'): '1',
 ('0', 'b'): '5',
 ('1', 'a'): '2',
 ('1', 'b'): '5',
 ('3', 'a'): '4',
 ('3', 'b'): '5',
 ('4', 'a'): '4',
 ('4', 'b'): '5',
 ('2', 'a'): '3',
 ('2', 'b'): '5',
 ('5', 'a'): '5',
 ('5', 'b'): '5'}

In [38]:
Automata5.f

['0', '1', '2', '4']

In [39]:
Automata5.proccess_word("a")

Valid word


In [40]:
Automata5.proccess_word("aaaabbbb")

Unvalid word


# Implementation 6

In [41]:
Automata6 = Automaton()

In [42]:
Automata6.load_from_file("data/automata6.txt")

In [43]:
Automata6.sigma

['a', 'b']

In [44]:
Automata6.delta

{('0', 'a'): '1',
 ('0', 'b'): '6',
 ('1', 'a'): '2',
 ('1', 'b'): '6',
 ('2', 'a'): '3',
 ('2', 'b'): '6',
 ('3', 'a'): '4',
 ('3', 'b'): '6',
 ('4', 'a'): '5',
 ('4', 'b'): '6',
 ('5', 'a'): '5',
 ('5', 'b'): '6',
 ('6', 'a'): '6',
 ('6', 'b'): '6'}

In [45]:
Automata6.f

['0', '1', '3', '5']

In [46]:
Automata6.proccess_word("aaa")

Valid word


In [47]:
Automata6.proccess_word("aa")

Unvalid word


# Implementation 7

In [48]:
Automata7 = Automaton()

In [49]:
Automata7.load_from_file("data/automata7.txt")

TypeError: q0 is not a subset of states q

In [ ]:
Automata8 = Automaton()

In [ ]:
Automata8.load_from_file("data/automata8.txt")